In [1]:
import urllib.request
import re
import os
import html
import csv


def open_page():
    # создает главную папку, если ее еще нет, и csv-файл
    directory = 'newspaper'
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = '/metadata.csv'
    path = directory + filename
    # в созданном csv-файле пишет имена столбцов
    with open(path, 'w', encoding='utf-8') as f:
        f.write('path\tauthor\theader\tcreated\tsphere\ttopic\tstyle\
                 \taudience_age\taudience_level\taudience_size\tsource\
                 \tpublication\tpubl_year\tmedium\tcountry\tregion\
                 \tlanguage\n')
    
    # ссылка на газету
    newspaper_Url = 'https://hron.ru/news/read/'
    
    # цикл для обработки каждой статьи 
    for i in range(55250, 56249):   
        pageUrl = newspaper_Url + str(i)
        try:
            page = urllib.request.urlopen(pageUrl)
            html = page.read().decode('utf-8')
        except:
            continue
            return
        page = str(pageUrl)
        search_meta(search_author(html), search_date(html),
                    search_header(html), pageUrl, path, directory,
                    article(html))
        
    return html

# создает директорию для каждой статьи в зависимости от года и месяца
# издательства, записывает чистый текст статьи и информацию о ней в 
# новый файл


def search_meta(author, date, header, pageUrl, path, directory, article):
    
    path_plain = directory + '\plain' #  адрес папки со всеми текстами
    if not os.path.exists(path_plain):
        os.makedirs(path_plain)
    year = str(''.join(date))[6:10]
    year_dirname = '%s' % year 
    path_year = os.path.join(path_plain, year_dirname)
    if not os.path.exists(path_year):
        os.makedirs(path_year)
    month = month = str(''.join(date))[3:5]
    month_dirname = '%s' % month
    path_month = os.path.join(path_year, month_dirname)
    if not os.path.exists(path_month):
        os.makedirs(path_month)
    
    art_name = 'article' + str(pageUrl)[-5:] + '.txt'
    art_path = os.path.join(path_month, art_name)
    info = '@au' + author + '\n@ti' + header + '\n@da' + date + '\n@topic' + \
           'None' + '\n@url' + pageUrl + '\n'
    with open(art_path, 'w', encoding='utf-8') as f:
        f.write(info)
    with open(art_path, 'a+', encoding='utf-8') as fl:
        fl.write(article)
        
    # записывает мета-значения для каждой статьи в csv-файл
    meta = '%s\t%s\t%s\t%s\tпублицистика\tNone\tнейтральный\tн-возраст\
            \tн-уровень\tгородская\t%s\tОрская хроника\t%s\tгазета\tРоссия\
            \tОренбургская область\tru\n'
    
    meta_string = meta % (art_path, author, header, date, pageUrl, 
                          str(''.join(date))[6:10])
    with open(path, 'a') as f:
        f.write(meta_string)

    
def search_author(html):  # находит имя автора
    author = re.findall(r'<a href="/author/single/\w+" title="(.*?)">', 
                        html)
    author = ''.join(author)
    if author == '':
        author = 'None'
    return author


def search_date(html):  # находит дату статьи
    date = re.findall(r'<li class="detail date">(.*?)</li>', html)
    date = ''.join(date)
    date = date.replace('-', '.')
    return date


def search_header(html):  # находит название статьи
    header = re.findall(
        r'<title>(.*?)Ежедневная городская газета Орская хроника</title>', 
        html)
    header = ''.join(header)
    return header


def article(html):  # находит и чистит текст статьи
    article = re.findall(
        r'<div class="text">\n(.*?)</div>', html)
    article = ''.join(str(x) for x in article)
    regTag = re.compile('<.*?>', re.DOTALL)
    article = regTag.sub('', article)
    regNBSP = re.compile('[&nbsp;]', re.DOTALL)
    article = regNBSP.sub('', article)
    return article


if __name__ == '__main__':
    open_page()